In [2]:
import torch
import torch.nn.utils.prune as prune
from ultralytics import YOLO

#### Tutorial for PyTorch pruning: https://towardsdatascience.com/how-to-prune-neural-networks-with-pytorch-ebef60316b91

In [18]:
def prune_model(model, amount=0.1, prune_type="unstructured", dim=0):
    """
    Prunes the model based on the specified prune_type.
    
    Args:
        model: The model to be pruned.
        amount: The amount of pruning to apply (e.g., fraction of weights to prune).
        prune_type: Type of pruning: "unstructured" or "structured".
        dim: Dimension along which to apply structured pruning (only for structured).
    
    Returns:
        model: The pruned model.
    """
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            print(f"Pruning module: {module}")
            
            if prune_type == "unstructured":
                # Apply unstructured pruning
                prune.l1_unstructured(module, name="weight", amount=amount)
            elif prune_type == "structured":
                # Apply structured pruning
                prune.ln_structured(module, name="weight", amount=amount, n=1, dim=dim)
            else:
                raise ValueError(f"Unsupported prune_type: {prune_type}")
            
            # Remove the pruning mask and update the weights
            prune.remove(module, "weight")
    
    return model

In [15]:
model=YOLO("../cocoa_diseases_yolo11x_dense/train6/weights/best.pt")
print(model.info())
torch_model=model.model

YOLO11x summary: 631 layers, 56,877,241 parameters, 0 gradients, 195.5 GFLOPs
(631, 56877241, 0, 195.46199040000002)


In [4]:
val_dense = model.val(
    data="../datasets/cocoa_diseases/cocoa_dataset.yaml",
    imgsz=640,
    batch=2,  # Small batch size
    device=[0,1]  # GPU
)

Ultralytics 8.3.58 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A16, 15000MiB)
                                                       CUDA:1 (NVIDIA A16, 15000MiB)
YOLO11x summary (fused): 464 layers, 56,830,489 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:04<00:00,  6.45it/s]


                   all         62        289      0.735      0.648       0.73      0.506
          phytophthora         30         39      0.781      0.564      0.678        0.5
               monilia         23         30      0.653      0.767      0.752       0.56
               healthy         52        220      0.771      0.613       0.76      0.457
Speed: 0.9ms preprocess, 67.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val


## Local unstructured pruning

In [15]:
pruned_torch_model=prune_model(torch_model,prune_type='unstructured', amount=0.3)
print("Model pruned.")
model.model=pruned_torch_model
model.save('yolo11x_trained_pruned_local_unstructured_30.pt')
print("Model saved.")

Model sparsity after pruning: 30.00%
Model pruned.


In [5]:
sparse_model=YOLO('yolo11x_trained_pruned_local_unstructured_30.pt')
sparse_model.info()

YOLO11x summary: 631 layers, 56,877,241 parameters, 0 gradients, 195.5 GFLOPs


(631, 56877241, 0, 195.46199040000002)

In [4]:
val_sparse= sparse_model.val(
    data="../datasets/cocoa_diseases/cocoa_dataset.yaml",
    imgsz=640,
    batch=2,  # Small batch size
    device=[0,1]  # GPU
)

Ultralytics 8.3.58 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A16, 15000MiB)
                                                       CUDA:1 (NVIDIA A16, 15000MiB)
YOLO11x summary (fused): 464 layers, 56,830,489 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:05<00:00,  5.87it/s]


                   all         62        289     0.0769      0.179     0.0393     0.0145
          phytophthora         30         39      0.108      0.205     0.0446     0.0165
               monilia         23         30      0.111        0.3     0.0675     0.0246
               healthy         52        220     0.0119     0.0318    0.00563    0.00241
Speed: 0.8ms preprocess, 62.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/val4


## Local structured pruning

### Channel-wise

In [13]:
pruned_torch_model_structured=prune_model(torch_model, prune_type='structured', dim=0, amount=0.3)
print(pruned_torch_model.info())
print("Model pruned.")
model.model=pruned_torch_model
model.save('yolo11x_trained_pruned_local_structured_30_channel.pt')
print("Model saved.")


YOLO11x summary: 631 layers, 56,877,241 parameters, 0 gradients, 195.5 GFLOPs
(631, 56877241, 0, 195.46199040000002)
Model pruned.
Model saved.


In [14]:
sparse_model=YOLO('yolo11x_trained_pruned_local_structured_30_channel.pt')
val_sparse= sparse_model.val(
    data="../datasets/cocoa_diseases/cocoa_dataset.yaml",
    imgsz=640,
    batch=2,  # Small batch size
    device=[0,1]  # GPU
)

Ultralytics 8.3.58 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A16, 15000MiB)
                                                       CUDA:1 (NVIDIA A16, 15000MiB)
YOLO11x summary (fused): 464 layers, 56,830,489 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:04<00:00,  7.15it/s]


                   all         62        289   0.000436    0.00758   0.000224   6.27e-05
          phytophthora         30         39          0          0          0          0
               monilia         23         30          0          0          0          0
               healthy         52        220    0.00131     0.0227   0.000671   0.000188
Speed: 0.9ms preprocess, 60.8ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val5


### Neuron-wise

In [16]:
pruned_torch_model=prune_model(torch_model, prune_type='structured', dim=1, amount=0.3)
print(pruned_torch_model.info())
print("Model pruned.")
model.model=pruned_torch_model
model.save('yolo11x_trained_pruned_local_structured_30_neuron.pt')
print("Model saved.")

YOLO11x summary: 631 layers, 56,877,241 parameters, 0 gradients, 195.5 GFLOPs
(631, 56877241, 0, 195.46199040000002)
Model pruned.
Model saved.


In [17]:
sparse_model=YOLO('yolo11x_trained_pruned_local_structured_30_neuron.pt')
val_sparse= sparse_model.val(
    data="../datasets/cocoa_diseases/cocoa_dataset.yaml",
    imgsz=640,
    batch=2,  # Small batch size
    device=[0,1]  # GPU
)

Ultralytics 8.3.58 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A16, 15000MiB)
                                                       CUDA:1 (NVIDIA A16, 15000MiB)
YOLO11x summary (fused): 464 layers, 56,830,489 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:04<00:00,  7.29it/s]

                   all         62        289          0          0          0          0


Speed: 1.0ms preprocess, 62.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val6
